In this you split the training dataset into various chunks say K chunks.

You use k-1 chunks for training purpose and the last chunk for testing.

The model performs in such a way that each chunk created is used for testing and in the end, average is taken of all

In [4]:

import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn import datasets, svm
from sklearn.datasets import load_digits

In [5]:
dg = load_digits()

X_train,X_test,y_train,y_test = train_test_split(dg.data, dg.target,test_size=0.2,random_state=2)

## Logistic Regression

In [11]:
from sklearn.linear_model import LogisticRegression
model_log = LogisticRegression(solver='liblinear',multi_class='ovr')
model_log.fit(X_train,y_train)
model_log.score(X_test,y_test)



0.9361111111111111

## SVM


In [15]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train,y_train)
svm.score(X_test,y_test)

0.9805555555555555

## Random Forest

In [17]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=40)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9555555555555556

## KFold Cross Validation

In [23]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
kf

KFold(n_splits=3, random_state=None, shuffle=False)

In [25]:
## lets see how kfold works on simple example

for train_index, test_index in kf.split([1,2,3,4,5,6,7,8,9]):
    print(train_index, test_index)

## see in the test_index, all the points are covered 
# 
# see also by chnaging the n_splits to 2   

[3 4 5 6 7 8] [0 1 2]
[0 1 2 6 7 8] [3 4 5]
[0 1 2 3 4 5] [6 7 8]


In [39]:
## model for getting the accuracy

def get_score(model,X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test,y_test)

Use stratifiedKfold when you want the same split to be taken into consideration when you are want to know about accuracy of diff. models

Three commonly used variations are as follows:


Train/Test Split: Taken to one extreme, k may be set to 2 (not 1) such that a single train/test split is created to evaluate the model.


LOOCV: Taken to another extreme, k may be set to the total number of observations in the dataset such that each observation is given a chance to be the held out of the dataset. This is called leave-one-out cross-validation, or LOOCV for short.


Stratified: The splitting of data into folds may be governed by criteria such as ensuring that each fold has the same proportion of observations with a given categorical value, such as the class outcome value. This is called stratified cross-validation.


Repeated: This is where the k-fold cross-validation procedure is repeated n times, where importantly, the data sample is shuffled prior to each repetition, which results in a different split of the sample.


Nested: This is where k-fold cross-validation is performed within each fold of cross-validation, often to perform hyperparameter tuning during model evaluation. This is called nested cross-validation or double cross-validation.

In [40]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=3)

scores_logistic =[]
scores_svm = []
scores_rf = []

for train_index, test_index in folds.split(dg.data, dg.target):
    X_train, X_test, y_train, y_test = dg.data[train_index,], dg.data[test_index], \
                                        dg.target[train_index], dg.target[test_index]
                            
    scores_logistic.append(get_score(model_log,X_train, X_test, y_train, y_test)) 
    scores_svm.append(get_score(svm,X_train, X_test, y_train, y_test)) 
    scores_rf.append(get_score(rf,X_train, X_test, y_train, y_test)) 
                   

In [42]:
print(scores_logistic)
print(scores_svm)
print(scores_rf)

[0.8948247078464107, 0.9532554257095158, 0.9098497495826378]
[0.9649415692821369, 0.9799666110183639, 0.9649415692821369]
[0.9382303839732888, 0.9432387312186978, 0.9315525876460768]


## Cross Validation Scores

In [46]:
from sklearn.model_selection import cross_val_score

## logistics reg
print(cross_val_score(model_log,dg.data,dg.target,cv =3))

## svm
print(cross_val_score(svm,dg.data,dg.target,cv =3))

## random forest
print(cross_val_score(rf,dg.data,dg.target,cv =3))

[0.89482471 0.95325543 0.90984975]
[0.96494157 0.97996661 0.96494157]
[0.92320534 0.94824708 0.92153589]


In [48]:
## lets see parametere tuning in RF

n = [10,20,30,50,80,100,150]

for i in n:
    scores2 = 0
    scores2 = cross_val_score(RandomForestClassifier(n_estimators = i),dg.data,dg.target,cv =10)
    print(i," ", np.average(scores2))

10   0.9254096834264433
20   0.9404469273743017
30   0.9404531346989445
50   0.9465735567970205
80   0.9471135940409683
100   0.9488050900062073
150   0.9465735567970205
